In [1]:
import re
from Bio import SeqIO
import requests

In [6]:
import requests

files = [
    "PBMC_1_con_sn.pride.mztab.gz",
    "PBMC_2_con_sn.pride.mztab.gz",
    "PBMC_3_con_sn.pride.mztab.gz"
]

base_url = "https://ftp.pride.ebi.ac.uk/pride/data/archive/2014/11/PXD001417/"

for fname in files:
    url = base_url + fname
    print(f"Downloading {fname}...")
    r = requests.get(url, stream=True)
    if r.status_code == 200:
        with open(fname, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    else:
        print(f"Failed to download {fname} (status code {r.status_code})")

ConnectTimeout: HTTPSConnectionPool(host='ftp.pride.ebi.ac.uk', port=443): Max retries exceeded with url: /pride/data/archive/2014/11/PXD001417/PBMC_1_con_sn.pride.mztab.gz (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f0e9811ffa0>, 'Connection to ftp.pride.ebi.ac.uk timed out. (connect timeout=None)'))

In [ ]:
import gzip
import shutil

for fname in files:
    with gzip.open(fname, 'rb') as f_in:
        with open(fname.replace(".gz", ""), 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [3]:
from Bio import SeqIO
import re

def tryptic_digest(seq, missed_cleavages=1, min_len=6, max_len=30):
    cleavage_sites = [0] + [m.end() for m in re.finditer(r'(?<=[KR])(?!P)', seq)] + [len(seq)]
    peptides = set()
    for i in range(len(cleavage_sites) - 1):
        for j in range(1, missed_cleavages + 2):
            if i + j < len(cleavage_sites):
                pep = seq[cleavage_sites[i]:cleavage_sites[i + j]]
                if min_len <= len(pep) <= max_len:
                    peptides.add(pep)
    return peptides

# Load protein FASTA
orf_peptides = {}
for record in SeqIO.parse("CMC1_transcripts.fa", "fasta"):
    peptides = tryptic_digest(str(record.seq))
    orf_peptides[record.id] = peptides

In [4]:
import pandas as pd

# Load MaxQuant peptides
df = pd.read_csv("peptides.txt", sep='\t')

# Filter
filtered = df[
    (df["Reverse"] != "+") &
    (df["Potential contaminant"] != "+") &
    (df["PEP"] <= 0.05)
]

observed_peptides = set(filtered["Sequence"])

FileNotFoundError: [Errno 2] No such file or directory: 'peptides.txt'